# Capstone: Trump vs Stock Market

# Data Collection Notebook

## Contents
- [Problem statement](#Problem-statement)
- [Data dictionary](#Data-dictionary)
- [Importing Trump twitter dataset](#Importing-Trump-twitter-dataset)
- [Data Cleaning](#Data-Cleaning)
- [Feature engineering](#Feature-engineering)
- [Identifying positive class](#Identifying-positive-class)
- [Train Validation and Holdout Data](#Train-Validation-and-Holdout-Data)

## Problem statement

The US market team at a local bank has seen literature on models that are able to predict market movement based on tweets by Donald Trump:
- [JP Morgan creating a 'Volfefe index to track tweets vs bond interest rates](https://www.marketwatch.com/story/are-trump-tweets-influencing-bond-volatility-jp-morgans-volfefe-index-aims-to-find-out-2019-09-09)
- [On days when President Trump tweets a lot, the stock market falls, Bank of America](https://www.cnbc.com/2019/09/03/on-days-when-president-trump-tweets-a-lot-the-stock-market-falls-investment-bank-finds.html)

They have tasked the data science team to build a classification model using Natural Language Processing to predict if Donald Trump's tweets are market moving. The models used are:
- Logistic Regression
- XGBoost
- Long Short Term Memory Neural Network
- Evaluate the models based on:
    - accuracy (% predictions the model gets correct, both a significant movement and a non-significant movement)
    - precision (% predicted significant movement when it is actually significant movement)
    - sensitivity (% predicted significant movement out of all correct predictions)
- choose the best performing model to test it on the holdout csv

### Datasets:
- Tweets from Donald Trump 2009 till June 2020
    - sourced from https://www.kaggle.com/austinreese/trump-tweets
    - 43352 recorded tweets since May 2009
- Historical S&P500 stock price from Yahoo Finance
- Loughran McDonald financial sentiment library
    - sourced from https://sraf.nd.edu/textual-analysis/resources/#Master%20Dictionary
    
### Assumptions made:
- Donald Trump's time of tweet is in the same time zone as the NYSE
- Stock market trading time is from 9:30am to 4pm
- He offically became president on 20th Jan 2017

## Data dictionary

### df_tweets
| Feature     	| Type 	| Description|                                                  
|:------------------	|:----------	|:----------------------------------------------------------------------------------|
| content	| object | Raw tweets from therealdonaldtrump |
| date	| object | Date of his tweet. YYYY-MM-DD-HH-MM-SS format |
| retweets	| int64 | Number of retweets by others at the time of data collection |
| favorites	| int64 | Number of favourites at the time of data collection |
| cleaned_tweets | object | Pre-processed tweets |
| tweet_day	| datetime64[ns] | Date of tweet YYYY-MM-DD format |
| month_sin	| float64 | Cycial sin month feature engineered. Range of -1 to 1 |
| month_cos	| float64 | Cycical cosine month feature engineered. Range of -1 to 1 |
| day_sin	| float64 | Cycical sin day feature engineered. Range of -1 to 1 |
| day_cos	| float64 | Cycical cosine day feature engineered. Range of -1 to 1 |
| hour_sin	| float64 | Cycical sin hour feature engineered. Range of -1 to 1 |
| hour_cos	| float64 | Cycical cosine hour feature engineered. Range of -1 to 1 |
| min_sin	| float64 | Cycical sin min feature engineered. Range of -1 to 1 |
| min_cos	| float64 | Cycical cosine min feature engineered. Range of -1 to 1 |
| time_to_open	| float64 | Time before the stock market opens on that day. In seconds | 	
| time_after_close	| float64 | Time after the stock market closes on that day. In seconds |	
| potus_status	| int64 | President of the United States status |
| vader_negative	| float64 | Negative sentiment analysis using Vader. Scale of 0 to 1 |
| vader_neutral	| float64 | Neutral sentiment analysis using Vader. Scale of 0 to 1 |
| vader_positive	| float64 | Positive sentiment analysis using Vader. Scale of 0 to 1 |
| vader_compound	| float64 | Overall sentiment analysis using Vader. Scale of 0 to 1 |	
| negative_word	| int64 | Negative word count using Loughran McDonald Financial sentiment analysis |
| positive_word	| int64 | Positive word count using Loughran McDonald Financial sentiment analysis |
| uncertainty_word	| int64 | Uncertainty word count using Loughran McDonald Financial sentiment analysis |
| litigious_word	| int64 | Litigious word count using Loughran McDonald Financial sentiment analysis |
| constraining_word	| int64 | Constraining word count using Loughran McDonald Financial sentiment analysis |
| interesting_word	| int64 | Interesting word count using Loughran McDonald Financial sentiment analysis |
| modal_strong_word	| int64 | Modal Strong word count using Loughran McDonald Financial sentiment analysis |
| modal_neutral_word	| int64 | Modal Neutral word count using Loughran McDonald Financial sentiment analysis |
| modal_weak_word	| int64 | Modal Weak word count using Loughran McDonald Financial sentiment analysis |
| difference	| float64 | Intra-day difference between opening and closing S&P500 stock price  	
| target	| int64 | Target variable |

In [1]:
# importing libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from progressbar import ProgressBar

# web scraping libraries
import re
import random
import requests
from bs4 import BeautifulSoup
from datetime import datetime, time

# NLP libraries
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#library for historical stock prices
import yfinance as yf
from datetime import timedelta
from dateutil.relativedelta import relativedelta

# Importing Trump twitter dataset

In [2]:
# importing the dataset
df_tweets = pd.read_csv('../datasets/realdonaldtrump.csv')

In [3]:
# checking first few rows
df_tweets.head()

id                                               link  \
0  1698308935  https://twitter.com/realDonaldTrump/status/169...   
1  1701461182  https://twitter.com/realDonaldTrump/status/170...   
2  1737479987  https://twitter.com/realDonaldTrump/status/173...   
3  1741160716  https://twitter.com/realDonaldTrump/status/174...   
4  1773561338  https://twitter.com/realDonaldTrump/status/177...   

                                             content                 date  \
0  Be sure to tune in and watch Donald Trump on L...  2009-05-04 13:54:25   
1  Donald Trump will be appearing on The View tom...  2009-05-04 20:00:10   
2  Donald Trump reads Top Ten Financial Tips on L...  2009-05-08 08:38:08   
3  New Blog Post: Celebrity Apprentice Finale and...  2009-05-08 15:40:15   
4  "My persona will never be that of a wallflower...  2009-05-12 09:07:28   

   retweets  favorites mentions hashtags  
0       510        917      NaN      NaN  
1        34        267      NaN      NaN  
2        13         19      NaN      NaN  
3        11         26      NaN      NaN  
4      1375       1945      NaN      NaN

In [4]:
# checking last few rows
df_tweets.tail()

id                                               link  \
43347  1273405198698975232  https://twitter.com/realDonaldTrump/status/127...   
43348  1273408026968457216  https://twitter.com/realDonaldTrump/status/127...   
43349  1273442195161387008  https://twitter.com/realDonaldTrump/status/127...   
43350  1273442469066276864  https://twitter.com/realDonaldTrump/status/127...   
43351  1273442528411385858  https://twitter.com/realDonaldTrump/status/127...   

                                                 content                 date  \
43347  Joe Biden was a TOTAL FAILURE in Government. H...  2020-06-17 19:00:32   
43348  Will be interviewed on @ seanhannity tonight a...  2020-06-17 19:11:47   
43349                         pic.twitter.com/3lm1spbU8X  2020-06-17 21:27:33   
43350                         pic.twitter.com/vpCE5MadUz  2020-06-17 21:28:38   
43351                         pic.twitter.com/VLlc0BHW41  2020-06-17 21:28:52   

       retweets  favorites      mentions hashtags  
43347     23402     116377           NaN      NaN  
43348     11810      56659  @seanhannity      NaN  
43349      4959      19344           NaN      NaN  
43350      4627      17022           NaN      NaN  
43351      3951      14344           NaN      NaN

In [5]:
# checking shape
df_tweets.shape

(43352, 8)

In [6]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43352 entries, 0 to 43351
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         43352 non-null  int64 
 1   link       43352 non-null  object
 2   content    43352 non-null  object
 3   date       43352 non-null  object
 4   retweets   43352 non-null  int64 
 5   favorites  43352 non-null  int64 
 6   mentions   20386 non-null  object
 7   hashtags   5583 non-null   object
dtypes: int64(3), object(5)
memory usage: 2.6+ MB


# Data Cleaning

### Missing values

In [7]:
# checking for missing values
df_tweets.isnull().sum()

id               0
link             0
content          0
date             0
retweets         0
favorites        0
mentions     22966
hashtags     37769
dtype: int64

In [8]:
#dropping the missing values
df_tweets.dropna(axis=1, inplace=True)

In [9]:
#also drop id and link columns as these are unncessary
df_tweets.drop(['id', 'link'], axis=1, inplace=True)

### Text Data Pre-Processing

In [10]:
# Instantiating Tokenizer, PorterStemmer and lemmatizer

tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [11]:
# loding NLTK stopwords
stopwords_nltk = set(stopwords.words('english'))

# loading in spacy stopwords
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

for i in stopwords_nltk:
    all_stopwords.add(i)

additional_stop_words = ['twitter', 'pic', 'https', 'com', 'http', 'www', 'bit', 'ly']    

for i in additional_stop_words:
    all_stopwords.add(i)

In [12]:
#taken from DSI-14 week 5.03 NLP startercode
def pre_processing(raw_text):

    # 1. Remove HTML.
    review_text = BeautifulSoup(raw_text).get_text()
    
    # 2. Remove non-letters.
    # regex modified from https://towardsdatascience.com/covfefe-nlp-do-trumps-tweets-move-the-stock-market-42a83ab17fea
    letters_only = re.sub('(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)', ' ', review_text)
    
    # 3. Convert to lower case, split into individual words.
    words = letters_only.lower().split()
    
    # 4. Searching a set is much faster than searching a list
    #    Adding in additional stopwords.
    stops = all_stopwords
    
    # 5. Remove stopwords.
    meaningful_words = [w for w in words if w not in stops]
    
    
    #5.5 lemmatizing of words
    meaningful_words = [lemmatizer.lemmatize(w) for w in words]
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return(" ".join(meaningful_words))

In [13]:
# Get the number of posts on the dataframe size.
total_tweets= df_tweets.shape[0]
print(f'There are {total_tweets} tweets')

There are 43352 tweets


In [14]:
# Initialize an empty list to hold the clean posts.
pbar = ProgressBar()
cleaned_tweets = []

for tweets in pbar(df_tweets['content']):
    cleaned_tweets.append(pre_processing(tweets))

 92% (39971 of 43352) |################  | Elapsed Time: 0:00:30 ETA:   0:00:02C:\Users\user\Anaconda3\lib\site-packages\bs4\__init__.py:312: UserWarning: "...." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % self._decode_markup(markup)
 97% (42183 of 43352) |################# | Elapsed Time: 0:00:32 ETA:   0:00:00C:\Users\user\Anaconda3\lib\site-packages\bs4\__init__.py:398: UserWarning: "https://www.whitehouse.gov/openingamerica/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  markup
100% (43352 of 43352) |##################| Elapsed Time: 0:00:33 Time:  0:00:33


In [15]:
#appending cleaned_tweets back to the df
df_tweets['cleaned_tweets'] = cleaned_tweets

# Feature engineering

### Date into cyclical features

credit to: http://blog.davidkaleko.com/feature-engineering-cyclical-features.html

In [16]:
# Extracting the only the date of the tweet
df_tweets['tweet_day'] = pd.to_datetime(df_tweets['date'])
df_tweets['tweet_day'] = df_tweets['tweet_day'].map(lambda x: x.date())
df_tweets['tweet_day'] = pd.to_datetime(df_tweets['tweet_day'])

In [17]:
#extracting the necessary features from date
df_tweets['month'] = df_tweets['date'].map(lambda x: int(x[5:7]))
df_tweets['day'] = df_tweets['date'].map(lambda x: int(x[8:10]))
df_tweets['hour'] = df_tweets['date'].map(lambda x: int(x[11:13]))
df_tweets['min'] = df_tweets['date'].map(lambda x: int(x[-5:-3]))

In [18]:
#cyclical features for month
df_tweets['month_sin'] = np.sin(df_tweets['month']*(2.*np.pi/12))
df_tweets['month_cos'] = np.cos(df_tweets['month']*(2.*np.pi/12))

#cyclical features for day
#assumption that every month has 31 days
df_tweets['day_sin'] = np.sin(df_tweets['day']*(2.*np.pi/31))
df_tweets['day_cos'] = np.cos(df_tweets['day']*(2.*np.pi/31))

#cyclical features for hour
df_tweets['hour_sin'] = np.sin(df_tweets['hour']*(2.*np.pi/24))
df_tweets['hour_cos'] = np.cos(df_tweets['hour']*(2.*np.pi/24))

#cyclical features for minute
df_tweets['min_sin'] = np.sin(df_tweets['min']*(2.*np.pi/60))
df_tweets['min_cos'] = np.cos(df_tweets['min']*(2.*np.pi/60))

In [19]:
df_tweets.head()

content                 date  \
0  Be sure to tune in and watch Donald Trump on L...  2009-05-04 13:54:25   
1  Donald Trump will be appearing on The View tom...  2009-05-04 20:00:10   
2  Donald Trump reads Top Ten Financial Tips on L...  2009-05-08 08:38:08   
3  New Blog Post: Celebrity Apprentice Finale and...  2009-05-08 15:40:15   
4  "My persona will never be that of a wallflower...  2009-05-12 09:07:28   

   retweets  favorites                                     cleaned_tweets  \
0       510        917  be sure to tune in and watch donald trump on l...   
1        34        267  donald trump will be appearing on the view tom...   
2        13         19  donald trump read top ten financial tip on lat...   
3        11         26  new blog post celebrity apprentice finale and ...   
4      1375       1945  my persona will never be that of a wallflower ...   

   tweet_day  month  day  hour  min  month_sin  month_cos   day_sin   day_cos  \
0 2009-05-04      5    4    13   54        0.5  -0.866025  0.724793  0.688967   
1 2009-05-04      5    4    20    0        0.5  -0.866025  0.724793  0.688967   
2 2009-05-08      5    8     8   38        0.5  -0.866025  0.998717 -0.050649   
3 2009-05-08      5    8    15   40        0.5  -0.866025  0.998717 -0.050649   
4 2009-05-12      5   12     9    7        0.5  -0.866025  0.651372 -0.758758   

   hour_sin  hour_cos   min_sin   min_cos  
0 -0.258819 -0.965926 -0.587785  0.809017  
1 -0.866025  0.500000  0.000000  1.000000  
2  0.866025 -0.500000 -0.743145 -0.669131  
3 -0.707107 -0.707107 -0.866025 -0.500000  
4  0.707107 -0.707107  0.669131  0.743145

In [20]:
# Extracting the time of tweet
df_tweets['tweet_time'] = pd.to_datetime(df_tweets['date'])
df_tweets['tweet_time'] = df_tweets['tweet_time'].map(lambda x: x.time())
df_tweets['tweet_time'] = pd.to_datetime(df_tweets['tweet_time'], format='%H:%M:%S') - pd.to_datetime(df_tweets['tweet_time'], format='%H:%M:%S').dt.normalize()

In [21]:
# new columns that measures how close the tweet is to opening/ closing
# market opens at 9:30am and closes at 4pm
market_open = timedelta(hours=9, minutes=30)
market_close = timedelta(hours=16)

def time_to_open(time):
    if time <= market_open:
        return market_open - time
    elif time > market_open:
        return timedelta(0)
    
def time_after_close(time):
    if time >= market_close:
        return time - market_close
    elif time < market_close:
        return timedelta(0)  


#converting to seconds    
df_tweets['time_to_open'] = df_tweets['tweet_time'].map(lambda x: time_to_open(x).total_seconds())
df_tweets['time_after_close'] = df_tweets['tweet_time'].map(lambda x: time_after_close(x).total_seconds())

In [22]:
df_tweets.head()

content                 date  \
0  Be sure to tune in and watch Donald Trump on L...  2009-05-04 13:54:25   
1  Donald Trump will be appearing on The View tom...  2009-05-04 20:00:10   
2  Donald Trump reads Top Ten Financial Tips on L...  2009-05-08 08:38:08   
3  New Blog Post: Celebrity Apprentice Finale and...  2009-05-08 15:40:15   
4  "My persona will never be that of a wallflower...  2009-05-12 09:07:28   

   retweets  favorites                                     cleaned_tweets  \
0       510        917  be sure to tune in and watch donald trump on l...   
1        34        267  donald trump will be appearing on the view tom...   
2        13         19  donald trump read top ten financial tip on lat...   
3        11         26  new blog post celebrity apprentice finale and ...   
4      1375       1945  my persona will never be that of a wallflower ...   

   tweet_day  month  day  hour  min  month_sin  month_cos   day_sin   day_cos  \
0 2009-05-04      5    4    13   54        0.5  -0.866025  0.724793  0.688967   
1 2009-05-04      5    4    20    0        0.5  -0.866025  0.724793  0.688967   
2 2009-05-08      5    8     8   38        0.5  -0.866025  0.998717 -0.050649   
3 2009-05-08      5    8    15   40        0.5  -0.866025  0.998717 -0.050649   
4 2009-05-12      5   12     9    7        0.5  -0.866025  0.651372 -0.758758   

   hour_sin  hour_cos   min_sin   min_cos tweet_time  time_to_open  \
0 -0.258819 -0.965926 -0.587785  0.809017   13:54:25           0.0   
1 -0.866025  0.500000  0.000000  1.000000   20:00:10           0.0   
2  0.866025 -0.500000 -0.743145 -0.669131   08:38:08        3112.0   
3 -0.707107 -0.707107 -0.866025 -0.500000   15:40:15           0.0   
4  0.707107 -0.707107  0.669131  0.743145   09:07:28        1352.0   

   time_after_close  
0               0.0  
1           14410.0  
2               0.0  
3               0.0  
4               0.0

### POTUS status

In [23]:
# Adding a president of the united states column
potus_day = pd.to_datetime('2017-01-20')
df_tweets['potus_status'] = df_tweets['tweet_day'].map(lambda x: 1 if x >= potus_day else 0)

In [24]:
#dropping unnecessary columns
df_tweets.drop(['month', 'day', 'hour', 'min', 'tweet_time'], axis=1, inplace=True)

### Vader Sentiment Analysis

In [25]:
#instantiating Vader
vader = SentimentIntensityAnalyzer()

In [26]:
#creating functions to pull the scores
def neg_score(text):
    score = vader.polarity_scores(text)
    return score['neg']

def neu_score(text):
    score = vader.polarity_scores(text)
    return score['neu']
    
def positive_score(text):
    score = vader.polarity_scores(text)
    return score['pos']
    
def compound_score(text):
    score = vader.polarity_scores(text)
    return score['compound']    

In [27]:
#adding the vader sentiment to the dataframe
df_tweets['vader_negative'] = df_tweets['content'].apply(neg_score)
df_tweets['vader_neutral'] = df_tweets['content'].apply(neu_score)
df_tweets['vader_positive'] = df_tweets['content'].apply(positive_score)
df_tweets['vader_compound'] = df_tweets['content'].apply(compound_score)

In [28]:
df_tweets.head()

content                 date  \
0  Be sure to tune in and watch Donald Trump on L...  2009-05-04 13:54:25   
1  Donald Trump will be appearing on The View tom...  2009-05-04 20:00:10   
2  Donald Trump reads Top Ten Financial Tips on L...  2009-05-08 08:38:08   
3  New Blog Post: Celebrity Apprentice Finale and...  2009-05-08 15:40:15   
4  "My persona will never be that of a wallflower...  2009-05-12 09:07:28   

   retweets  favorites                                     cleaned_tweets  \
0       510        917  be sure to tune in and watch donald trump on l...   
1        34        267  donald trump will be appearing on the view tom...   
2        13         19  donald trump read top ten financial tip on lat...   
3        11         26  new blog post celebrity apprentice finale and ...   
4      1375       1945  my persona will never be that of a wallflower ...   

   tweet_day  month_sin  month_cos   day_sin   day_cos  hour_sin  hour_cos  \
0 2009-05-04        0.5  -0.866025  0.724793  0.688967 -0.258819 -0.965926   
1 2009-05-04        0.5  -0.866025  0.724793  0.688967 -0.866025  0.500000   
2 2009-05-08        0.5  -0.866025  0.998717 -0.050649  0.866025 -0.500000   
3 2009-05-08        0.5  -0.866025  0.998717 -0.050649 -0.707107 -0.707107   
4 2009-05-12        0.5  -0.866025  0.651372 -0.758758  0.707107 -0.707107   

    min_sin   min_cos  time_to_open  time_after_close  potus_status  \
0 -0.587785  0.809017           0.0               0.0             0   
1  0.000000  1.000000           0.0           14410.0             0   
2 -0.743145 -0.669131        3112.0               0.0             0   
3 -0.866025 -0.500000           0.0               0.0             0   
4  0.669131  0.743145        1352.0               0.0             0   

   vader_negative  vader_neutral  vader_positive  vader_compound  
0             0.0          0.827           0.173          0.5255  
1             0.0          0.749           0.251          0.7712  
2             0.0          0.739           0.261          0.6468  
3             0.0          1.000           0.000          0.0000  
4             0.0          1.000           0.000          0.0000

### Loughran McDonald Financial sentiment analysis

In [30]:
#loading the financial statement dictionary
df_sentiment = pd.read_csv('../datasets/LoughranMcDonald_MasterDictionary_2018.csv')

In [31]:
#creating dummies for modal
df_sentiment = pd.get_dummies(df_sentiment, columns = ['Modal'], drop_first=True)

In [32]:
#checking the df
df_sentiment.head()

Word  Sequence Number  Word Count  Word Proportion  \
0   AARDVARK                1         277     1.480000e-08   
1  AARDVARKS                2           3     1.600000e-10   
2      ABACI                3           8     4.280000e-10   
3      ABACK                4          12     6.410000e-10   
4     ABACUS                5        7250     3.870000e-07   

   Average Proportion       Std Dev  Doc Count  Negative  Positive  \
0        1.240000e-08  3.560000e-06         84         0         0   
1        9.730000e-12  9.860000e-09          1         0         0   
2        1.390000e-10  6.230000e-08          7         0         0   
3        3.160000e-10  9.380000e-08         12         0         0   
4        3.680000e-07  3.370000e-05        914         0         0   

   Uncertainty  Litigious  Constraining  Superfluous  Interesting  Irr_Verb  \
0            0          0             0            0            0         0   
1            0          0             0            0            0         0   
2            0          0             0            0            0         0   
3            0          0             0            0            0         0   
4            0          0             0            0            0         0   

   Harvard_IV  Syllables     Source  Modal_1  Modal_2  Modal_3  
0           0          2  12of12inf        0        0        0  
1           0          2  12of12inf        0        0        0  
2           0          3  12of12inf        0        0        0  
3           0          2  12of12inf        0        0        0  
4           0          3  12of12inf        0        0        0

In [33]:
#Renaming column headers using .rename takes significantly longer
#df_sentiment.rename({'Modal_1':'modal_strong', 'Modal_2':'modal_neutral', 'Modal_3':'modal_weak'}, axis=1)

df_sentiment.columns = ['Word', 'Sequence Number', 'Word Count', 'Word Proportion',
       'Average Proportion', 'Std Dev', 'Doc Count', 'Negative', 'Positive',
       'Uncertainty', 'Litigious', 'Constraining', 'Superfluous',
       'Interesting', 'Irr_Verb', 'Harvard_IV', 'Syllables', 'Source',
       'modal_strong', 'modal_neutral', 'modal_weak']

In [34]:
#Picking out the sentiment topics to use
sentiments = ['negative', 'positive', 'uncertainty', 'litigious', 'constraining', 
              'interesting', 'modal_strong', 'modal_neutral', 'modal_weak']
df_sentiment.columns = [column.lower() for column in df_sentiment.columns]

In [35]:
# changing the dictionary to only include words that appear in sentiments
df_sentiment = df_sentiment[['word'] + sentiments]
df_sentiment[sentiments] = df_sentiment[sentiments].astype(bool)
df_sentiment = df_sentiment[(df_sentiment[sentiments]).any(1)]

In [36]:
# Initialize an empty list to hold the clean posts.
clean_words = []

for words in df_sentiment['word']:
    
    # Convert post to words, then append to clean_train_posts.
    clean_words.append(pre_processing(words))

df_sentiment['word'] = clean_words
df_sentiment = df_sentiment.drop_duplicates('word')

In [37]:
#negative words
def negative_word(text):
    negative = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['negative']==1]['word']):
            negative += 1
    return negative

#positive words
def positive_word(text):
    positive = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['positive']==1]['word']):
            positive += 1
    return positive

#uncertainty words
def uncertainty_word(text):
    uncertainty = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['uncertainty']==1]['word']):
            uncertainty += 1
    return uncertainty

#litigious words
def litigious_word(text):
    litigious = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['litigious']==1]['word']):
            litigious += 1
    return litigious

#constraining words
def constraining_word(text):
    constraining = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['constraining']==1]['word']):
            constraining += 1
    return constraining

#interesting words
def interesting_word(text):
    interesting = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['interesting']==1]['word']):
            interesting += 1
    return interesting

#modal_strong words
def modal_strong_word(text):
    modal_strong = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['modal_strong']==1]['word']):
            modal_strong += 1
    return modal_strong

#modal_neutral words
def modal_neutral_word(text):
    modal_neutral = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['modal_neutral']==1]['word']):
            modal_neutral += 1
    return modal_neutral

#modal_weak words
def modal_weak_word(text):
    modal_weak = 0
    for word in tokenizer.tokenize(text.lower()):
        if word in set(df_sentiment[df_sentiment['modal_weak']==1]['word']):
            modal_weak += 1
    return modal_weak

functions = [negative_word, positive_word, uncertainty_word, litigious_word, constraining_word, 
             interesting_word, modal_strong_word, modal_neutral_word, modal_weak_word]

In [38]:
df_dict = df_tweets.copy()

In [39]:
# apply functions to the new coloumns
''' Commenting out as these take a long time to run'''
# df_dict['negative_word'] = df_dict['cleaned_tweets'].apply(negative_word)
# df_dict['positive_word'] = df_dict['cleaned_tweets'].apply(positive_word)
# df_dict['uncertainty_word'] = df_dict['cleaned_tweets'].apply(uncertainty_word)
# df_dict['litigious_word'] = df_dict['cleaned_tweets'].apply(litigious_word)
# df_dict['constraining_word'] = df_dict['cleaned_tweets'].apply(constraining_word)
# df_dict['interesting_word'] = df_dict['cleaned_tweets'].apply(interesting_word)
# df_dict['modal_strong_word'] = df_dict['cleaned_tweets'].apply(modal_strong_word)
# df_dict['modal_neutral_word'] = df_dict['cleaned_tweets'].apply(modal_neutral_word)
# df_dict['modal_weak_word'] = df_dict['cleaned_tweets'].apply(modal_weak_word)

' Commenting out as these take a long time to run'

In [40]:
# # saving
# df_dict[['negative_word','positive_word','uncertainty_word', 'litigious_word', 'constraining_word','interesting_word','modal_strong_word', 'modal_neutral_word', 'modal_weak_word']].to_csv('../datasets/df_dict.csv')

In [41]:
df_dict = pd.read_csv('../datasets/df_dict.csv', index_col=0)

In [42]:
df_dict.head()

negative_word  positive_word  uncertainty_word  litigious_word  \
0              1              0                 0               0   
1              0              0                 1               0   
2              1              0                 0               0   
3              0              0                 0               0   
4              0              0                 0               0   

   constraining_word  interesting_word  modal_strong_word  modal_neutral_word  \
0                  0                 0                  0                   0   
1                  0                 0                  1                   0   
2                  0                 0                  0                   0   
3                  0                 0                  0                   0   
4                  0                 0                  2                   0   

   modal_weak_word  
0                0  
1                1  
2                0  
3                0  
4                0

In [43]:
df_tweets = df_tweets.join(df_dict)

# Identifying positive class

### Adding the opening and closing price of the S&P500

In [44]:
# Ceating a dataframe of the S&P500 historical returns
df_stock_price = yf.Ticker('^GSPC').history(start='2009-05-01',end='2020-06-20')

In [45]:
# checking shape
df_stock_price.shape

(2805, 7)

In [46]:
# checking df
df_stock_price.head()

Open    High     Low   Close      Volume  Dividends  \
Date                                                                
2009-04-30  876.59  888.70  868.51  872.81  6862540000          0   
2009-05-01  872.74  880.48  866.10  877.52  5312170000          0   
2009-05-04  879.21  907.85  879.21  907.24  7038840000          0   
2009-05-05  906.10  907.70  897.34  903.80  6882860000          0   
2009-05-06  903.95  920.28  903.95  919.53  8555040000          0   

            Stock Splits  
Date                      
2009-04-30             0  
2009-05-01             0  
2009-05-04             0  
2009-05-05             0  
2009-05-06             0

In [47]:
# finding the S&P500 return within the day
df_stock_price['difference'] = (df_stock_price['Close'] - df_stock_price['Open'])/df_stock_price['Open']

In [48]:
df_stock_price.to_csv('../datasets/df_stock_price.csv')

In [49]:
df_stock_price.describe()

Open         High          Low        Close        Volume  \
count  2805.000000  2805.000000  2805.000000  2805.000000  2.805000e+03   
mean   1947.225597  1957.165786  1936.463836  1947.679084  3.874781e+09   
std     643.904533   646.016805   641.199932   643.627539  1.002020e+09   
min     872.740000   880.480000   866.100000   872.810000  1.025000e+09   
25%    1337.390000  1343.330000  1330.030000  1337.770000  3.275080e+09   
50%    1972.730000  1981.800000  1962.420000  1972.740000  3.665340e+09   
75%    2469.120000  2477.960000  2460.290000  2472.100000  4.240570e+09   
max    3380.450000  3393.520000  3378.830000  3386.150000  1.061781e+10   

       Dividends  Stock Splits   difference  
count     2805.0        2805.0  2805.000000  
mean         0.0           0.0     0.000359  
std          0.0           0.0     0.009356  
min          0.0           0.0    -0.065934  
25%          0.0           0.0    -0.003149  
50%          0.0           0.0     0.000554  
75%          0.0           0.0     0.004611  
max          0.0           0.0     0.054876

In [50]:
difference_mean = 0.000359
std_1 = 0.009356
std_2 = 2*0.009356
std_3 = 3*0.009356

In [51]:
# joining the df onto df_tweets
df_tweets = df_tweets.join(df_stock_price['difference'], on='tweet_day')

In [52]:
# Positive class as 1 standard deviation away from the mean difference
df_tweets['target'] = df_tweets['difference'].map(lambda x: 1 if x > difference_mean+std_1 or x < difference_mean-std_1 else 0)

In [53]:
# # positive class as gain in stock market, and negative class as loss in stock market
# df_tweets['target'] = df_tweets['difference'].map(lambda x: 1 if x >= 0 else 0)

In [54]:
df_tweets['target'].value_counts(normalize=True)

0    0.846282
1    0.153718
Name: target, dtype: float64

In [55]:
df_tweets.head()

content                 date  \
0  Be sure to tune in and watch Donald Trump on L...  2009-05-04 13:54:25   
1  Donald Trump will be appearing on The View tom...  2009-05-04 20:00:10   
2  Donald Trump reads Top Ten Financial Tips on L...  2009-05-08 08:38:08   
3  New Blog Post: Celebrity Apprentice Finale and...  2009-05-08 15:40:15   
4  "My persona will never be that of a wallflower...  2009-05-12 09:07:28   

   retweets  favorites                                     cleaned_tweets  \
0       510        917  be sure to tune in and watch donald trump on l...   
1        34        267  donald trump will be appearing on the view tom...   
2        13         19  donald trump read top ten financial tip on lat...   
3        11         26  new blog post celebrity apprentice finale and ...   
4      1375       1945  my persona will never be that of a wallflower ...   

   tweet_day  month_sin  month_cos   day_sin   day_cos  hour_sin  hour_cos  \
0 2009-05-04        0.5  -0.866025  0.724793  0.688967 -0.258819 -0.965926   
1 2009-05-04        0.5  -0.866025  0.724793  0.688967 -0.866025  0.500000   
2 2009-05-08        0.5  -0.866025  0.998717 -0.050649  0.866025 -0.500000   
3 2009-05-08        0.5  -0.866025  0.998717 -0.050649 -0.707107 -0.707107   
4 2009-05-12        0.5  -0.866025  0.651372 -0.758758  0.707107 -0.707107   

    min_sin   min_cos  time_to_open  time_after_close  potus_status  \
0 -0.587785  0.809017           0.0               0.0             0   
1  0.000000  1.000000           0.0           14410.0             0   
2 -0.743145 -0.669131        3112.0               0.0             0   
3 -0.866025 -0.500000           0.0               0.0             0   
4  0.669131  0.743145        1352.0               0.0             0   

   vader_negative  vader_neutral  vader_positive  vader_compound  \
0             0.0          0.827           0.173          0.5255   
1             0.0          0.749           0.251          0.7712   
2             0.0          0.739           0.261          0.6468   
3             0.0          1.000           0.000          0.0000   
4             0.0          1.000           0.000          0.0000   

   negative_word  positive_word  uncertainty_word  litigious_word  \
0              1              0                 0               0   
1              0              0                 1               0   
2              1              0                 0               0   
3              0              0                 0               0   
4              0              0                 0               0   

   constraining_word  interesting_word  modal_strong_word  modal_neutral_word  \
0                  0                 0                  0                   0   
1                  0                 0                  1                   0   
2                  0                 0                  0                   0   
3                  0                 0                  0                   0   
4                  0                 0                  2                   0   

   modal_weak_word  difference  target  
0                0    0.031881       1  
1                1    0.031881       1  
2                0    0.022221       1  
3                0    0.022221       1  
4                0   -0.002383       0

# Train Validation and Holdout Data

In [56]:
df_tweets.to_csv('../datasets/df_tweets.csv')

In [57]:
df_tweets[df_tweets['cleaned_tweets']== '']

content                 date  \
3338                     @sayhigreg http://fxn.ws/VdRXwh  2012-09-20 15:16:36   
3339                . @ janinegibsonhttp://fxn.ws/VdRXwh  2012-09-20 15:19:44   
6874                                       @JaylonSeales  2013-02-27 01:39:28   
22171     # MakeAmericaGreatAgainhttp://on.fb.me/1zMGDu5  2015-04-22 12:49:22   
24953  # CNNDebatehttp://time.com/4037510/poll-second...  2015-09-16 23:10:37   
29512  # MakeAmericaGreatAgainhttps://twitter.com/emi...  2016-07-07 15:06:17   
29801  # CrookedHillaryhttp://www.dailymail.co.uk/new...  2016-08-01 09:48:22   
30578  # DrainTheSwamphttp://thehill.com/blogs/pundit...  2016-10-18 20:00:40   
30737  # ObamacareFailhttp://thehill.com/policy/healt...  2016-10-25 10:21:37   
31872  # MakeAmericaGreatAgainhttps://twitter.com/i/m...  2017-06-02 13:30:41   
33939  https://www.pscp.tv/w/baFQbzFvTlFsTFJub1dwUXd8...  2018-04-13 20:31:56   
34146  # USEmbassyJerusalemhttp://pscp.tv/w/1kvJpWoqe...  2018-05-14 10:16:38   
34495  https://www.pscp.tv/w/bfqHozFvTlFsTFJub1dwUXd8...  2018-06-20 12:37:42   
35110  https://www.pscp.tv/w/bk25IzFvTlFsTFJub1dwUXd8...  2018-08-22 15:05:05   
35470  https://www.pscp.tv/w/bnpJZDFvTlFsTFJub1dwUXd8...  2018-09-25 10:27:35   
35491  https://www.pscp.tv/w/boAVZDFvTlFsTFJub1dwUXd8...  2018-09-29 19:51:22   
35501  https://www.pscp.tv/w/boKyTTFvTlFsTFJub1dwUXd8...  2018-10-01 19:25:44   
35557  https://www.pscp.tv/w/boviwzFvTlFsTFJub1dwUXd8...  2018-10-08 18:43:34   
35569  https://www.pscp.tv/w/bo1KczFvTlFsTFJub1dwUXd8...  2018-10-09 20:18:12   
35712  https://www.pscp.tv/w/bp5vLjFvTlFsTFJub1dwUXd8...  2018-10-22 20:23:33   
36072  # USMCAhttps://www.pscp.tv/w/btEs2TFvTlFsTFJub...  2018-11-30 07:08:27   
36413  https://www.pscp.tv/w/bwVQxDFvTlFsTFJub1dwUXd8...  2019-01-08 20:16:29   
36513  https://www.pscp.tv/w/bxOMsjFvTlFsTFJub1dwUXd8...  2019-01-19 15:24:23   
36573  https://www.pscp.tv/w/bxtdRTFvTlFsTFJub1dwUXd8...  2019-01-25 13:39:28   
36652  # SOTUhttps://www.pscp.tv/w/bypNpDFvTlFsTFJub1...  2019-02-05 21:40:40   
36708  https://www.washingtonexaminer.com/opinion/its...  2019-02-13 18:41:14   
36839  https://www.pscp.tv/w/b0snXjFvTlFsTFJub1dwUXd8...  2019-03-02 19:39:36   
36948  https://www.pscp.tv/w/b1sFjTFvTlFsTFJub1dwUXd8...  2019-03-14 21:31:13   
37044  https://www.pscp.tv/w/b2kGljFvTlFsTFJub1dwUXd8...  2019-03-25 12:26:24   
37417  https://www.wsj.com/articles/a-real-attorney-g...  2019-05-01 20:58:10   
37679  https://www.pscp.tv/w/b7V2nTFvTlFsTFJub1dwUXd8...  2019-05-22 11:57:35   
37823  https://www.pscp.tv/w/b8Zu2zFvTlFsTFJub1dwUXd8...  2019-06-04 08:52:02   
37843  # DDay75thAnniversaryhttps://www.pscp.tv/w/b8j...  2019-06-06 06:50:25   
38144  https://www.doi.gov/pressreleases/what-expect-...  2019-07-02 11:29:07   
38763  https://www.pscp.tv/w/cCX70TFvTlFsTFJub1dwUXd8...  2019-08-15 20:18:43   
38923  https://www.pscp.tv/w/cDQERjFvTlFsTFJub1dwUXd8...  2019-08-26 11:50:37   
39037  https://www.whitehouse.gov/presidential-action...  2019-09-02 12:56:35   
39338  https://www.pscp.tv/w/cFewizFvTlFsTFJub1dwUXd8...  2019-09-22 13:11:17   
39384  https://www.pscp.tv/w/cFuHQTFvTlFsTFJub1dwUXd8...  2019-09-25 11:06:15   
39389  https://www.pscp.tv/w/cFvUmjFvTlFsTFJub1dwUXd8...  2019-09-25 16:34:00   
39427  https://www.whitehouse.gov/presidential-action...  2019-09-27 21:03:00   
39901  https://www.donaldjtrump.com/events/ms-2019-no...  2019-10-19 16:15:17   
39958  https://www.pscp.tv/w/cIBxZDFvTlFsTFJub1dwUXd8...  2019-10-23 11:06:00   
40006                                               ....  2019-10-26 07:18:29   
40018  https://www.pscp.tv/w/cIWhpTFvTlFsTFJub1dwUXd8...  2019-10-27 09:31:33   
40186  https://www.pscp.tv/w/cJOByDEzNTg4Mzh8MU93eFdk...  2019-11-06 21:09:29   
40236  https://www.donaldjtrump.com/events/bossier-ci...  2019-11-10 20:08:36   
40365  https://www.donaldjtrump.com/events/sunrise-fl...  2019-11-18 13:45:16   
40489  https://www.donaldjtrump.com/events/hershey-pa...  2019-11-30 14:06:19   

In [58]:
df_tweets = df_tweets[df_tweets['cleaned_tweets'] != '']

## Holdout Data

In [59]:
X = df_tweets.drop('target', axis=1)
y = df_tweets['target']

In [60]:
# Holdout data split
X_train_validate, X_holdout, y_train_validate, y_holdout = train_test_split(X,y,stratify=y, random_state=42, test_size=0.2)

In [61]:
X_holdout.shape

(8649, 31)

In [62]:
# Saving holdout data to csv
X_holdout.to_csv('../datasets/X_holdout.csv')
y_holdout.to_csv('../datasets/y_holdout.csv')

## Train and Validation

In [63]:
df_train_validate = X_train_validate.join(y_train_validate)

In [64]:
df_train_validate.head()

content                 date  \
24974  " @ LeghanLiptak712: CNN tried to destroy Dona...  2015-09-20 04:27:54   
17951  " @ InsideCableNews @ FTVLive Dream on. You ha...  2014-10-20 18:59:09   
1138   "The American work ethic is what led generatio...  2012-01-13 13:48:23   
17957  Wow, one of the all-time greats in fashion, OS...  2014-10-21 04:20:56   
32423  Received a # HurricaneHarvey briefing this mor...  2017-08-25 11:02:33   

       retweets  favorites                                     cleaned_tweets  \
24974      1123       2400  leghanliptak cnn tried to destroy donald trump...   
17951        17         38  insidecablenews ftvlive dream on you have a be...   
1138         93         40  the american work ethic is what led generation...   
17957       157        192  wow one of the all time great in fashion oscar...   
32423      6920      34234  received a hurricaneharvey briefing this morni...   

       tweet_day  month_sin     month_cos   day_sin   day_cos  hour_sin  \
24974 2015-09-20  -1.000000 -1.836970e-16 -0.790776 -0.612106  0.866025   
17951 2014-10-20  -0.866025  5.000000e-01 -0.790776 -0.612106 -1.000000   
1138  2012-01-13   0.500000  8.660254e-01  0.485302 -0.874347 -0.258819   
17957 2014-10-21  -0.866025  5.000000e-01 -0.897805 -0.440394  0.866025   
32423 2017-08-25  -0.866025 -5.000000e-01 -0.937752  0.347305  0.258819   

           hour_cos   min_sin   min_cos  time_to_open  time_after_close  \
24974  5.000000e-01  0.309017 -0.951057       18126.0               0.0   
17951 -1.836970e-16 -0.104528  0.994522           0.0           10749.0   
1138  -9.659258e-01 -0.951057  0.309017           0.0               0.0   
17957  5.000000e-01  0.866025 -0.500000       18544.0               0.0   
32423 -9.659258e-01  0.207912  0.978148           0.0               0.0   

       potus_status  vader_negative  vader_neutral  vader_positive  \
24974             0           0.078          0.658           0.264   
17951             0           0.152          0.638           0.210   
1138              0           0.000          0.755           0.245   
17957             0           0.068          0.653           0.279   
32423             1           0.000          1.000           0.000   

       vader_compound  negative_word  positive_word  uncertainty_word  \
24974          0.8113              2              0                 0   
17951          0.2263              1              2                 0   
1138           0.6369              0              1                 0   
17957          0.7953              0              3                 0   
32423          0.0000              0              0                 0   

       litigious_word  constraining_word  interesting_word  modal_strong_word  \
24974               0                  0                 0                  0   
17951               0                  0                 0                  0   
1138                0                  0                 0                  0   
17957               0                  0                 1                  0   
32423               0                  0                 0                  0   

       modal_neutral_word  modal_weak_word  difference  target  
24974                   0                0         NaN       0  
17951                   0                0    0.009753       1  
1138                    0                0   -0.004425       0  
17957                   0                0    0.016707       1  
32423                   0                0   -0.000683       0

In [65]:
df_train_validate.to_csv('../datasets/df_train_validate.csv')